In [57]:
# set workspace
WhoIsWorking = "AI"
WorkLocation = "Home"

if WhoIsWorking == "RT":
    if WorkLocation == "Home":
        folder_path = r'C:\Users\rentr\Desktop\ENGO\DataMining\ENGO645_TermProject_Data'
    elif WorkLocation == "School":
        folder_path = r'D:\ENGO645_TermProject_Data'
elif WhoIsWorking == "AA":
    folder_path = ''
elif WhoIsWorking == "IA":
    folder_path = r'C:\Users\afoam\assignments\ENGO645_TorontoCrimeAnalysis'
elif WhoIsWorking == "AI":
    folder_path = r'C:\Adewale_Directory\Capacity_Building\MGIS\Lecture_Note\WINTER2024\ENGO645\Course_Project\Project_Data'

print(folder_path)

C:\Adewale_Directory\Capacity_Building\MGIS\Lecture_Note\WINTER2024\ENGO645\Course_Project\Project_Data


In [58]:
#Install and import neccessary libraries
import os
import numpy as np
import geopandas as gpd
import pandas as pd
from libpysal.weights import lat2W
from esda.moran import Moran

In [59]:

# Load and read the MCI file
MCI_file = os.path.join(folder_path, "MCI_Cleaned.csv") # Update to the actual file location
MCI_data = pd.read_csv(MCI_file)
 
MCI_data #display dataframe

,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_DOY,OCC_DOW,OCC_HOUR,LOCATION_TYPE,PREMISES_TYPE,OFFENCE,MCI_CATEGORY,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,DATE_TIME_LINK,TIMECategory,TEMP,TEMPCategory
0,2023,January,1,1,Sunday,2,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery With Weapon,Robbery,Corso Italia-Davenport,-79.450529,43.676205,2023/1/1 2,Night,4.7,Mild
1,2023,January,1,1,Sunday,1,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery - Other,Robbery,St Lawrence-East Bayfront-The Islands,-79.368457,43.648776,2023/1/1 1,Night,4.5,Mild
2,2023,January,1,1,Sunday,9,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Morningside Heights,-79.202638,43.815274,2023/1/1 9,Morning,4.4,Mild
3,2023,January,1,1,Sunday,3,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Assault With Weapon,Assault,New Toronto,-79.517052,43.600915,2023/1/1 3,Night,4.9,Mild
4,2023,January,1,1,Sunday,21,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Theft Of Motor Vehicle,Auto Theft,Victoria Village,-79.300571,43.719703,2023/1/1 21,Evening,4.1,Mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365681,2014,December,31,365,Wednesday,16,"Apartment (Rooming House, Condo)",Apartment,B&E,Break and Enter,Wellington Place,-79.390275,43.647849,2014/12/31 16,Afternoon,-6.3,Chilly
365682,2014,December,31,365,Wednesday,12,"Single Home, House (Attach Garage, Cottage, Mo...",House,Assault With Weapon,Assault,Bendale-Glen Andrew,-79.248690,43.761776,2014/12/31 12,Afternoon,-6.3,Chilly
365683,2014,December,31,365,Wednesday,12,"Single Home, House (Attach Garage, Cottage, Mo...",House,Assault,Assault,Bendale-Glen Andrew,-79.248690,43.761776,2014/12/31 12,Afternoon,-6.3,Chilly
365684,2014,December,31,365,Wednesday,22,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Greenwood-Coxwell,-79.325929,43.671381,2014/12/31 22,Evening,-5.7,Chilly


In [60]:
# Find the count of offenses by neighborhood to aggregate data to Neighbourhood level
neighbourhood_counts = MCI_data['NEIGHBOURHOOD_158'].value_counts().reset_index()
neighbourhood_counts.columns = ['NeighbourhoodName', 'Crime_Count']  # Rename columns

neighbourhood_counts

,NeighbourhoodName,Crime_Count
0,West Humber-Clairville,10309
1,Moss Park,8621
2,Downtown Yonge East,7762
3,Yonge-Bay Corridor,7294
4,York University Heights,7237
...,...,...
154,Maple Leaf,734
155,Guildwood,680
156,Woodbine-Lumsden,649
157,Lambton Baby Point,585


In [61]:
# Load and read the Neighbourhood file
neighbourhood_file = os.path.join(folder_path, "Neighbourhood_Cleaned.csv")
neighbourhood_data = pd.read_csv(neighbourhood_file)

In [62]:
# Join the aggregated crime counts with neighborhood data based on the common neighborhood field
neighbourhood_merged = pd.merge(neighbourhood_data, neighbourhood_counts, on='NeighbourhoodName', how='left')

# Drop rows with missing values (if any)
neighbourhood_merged = neighbourhood_merged.dropna(subset=['Crime_Count'])

In [63]:
# Check the shape and info of the data again
print(neighbourhood_merged.shape)
neighbourhood_merged.info()

(158, 13)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   geometry                    158 non-null    object 
 1   NeighbourhoodName           158 non-null    object 
 2   Poles_Count                 158 non-null    int64  
 3   POI_Count                   158 non-null    int64  
 4   TCamera_Count               158 non-null    int64  
 5   TotalPopulation             158 non-null    int64  
 6   MedianTotalIncome           158 non-null    int64  
 7   AverageTotalIncome          158 non-null    int64  
 8   BuildingCoveragePERCENTAGE  158 non-null    float64
 9   ParksOSNAPERCENTAGE         158 non-null    float64
 10  BuiltUpPERCENTAGE           158 non-null    float64
 11  AveragePrice                158 non-null    int64  
 12  Crime_Count                 158 non-null    int64  
dtypes: float64(3), int64(8), 

In [64]:
# Convert the DataFrame to a GeoDataFrame
neighbourhood_gdf = gpd.GeoDataFrame(neighbourhood_merged, geometry='geometry')

# Print the GeoDataFrame to verify
display(neighbourhood_gdf.head())


TypeError: Input must be valid geometry objects: POLYGON ((-79.3863510515018 43.6978312650188, -79.3862291956072 43.6975037220317, -79.38638812454001 43.6975077326547, -79.3864562999666 43.6975192590321, -79.3866606404473 43.6975010721977, -79.3870109672437 43.6974685463247, -79.38764557960261 43.6973977736555, -79.3895282148369 43.6969911842496, -79.3927549645318 43.6962824893798, -79.39502109634751 43.6958404334629, -79.39512850188009 43.695814734359, -79.39528335080389 43.6957799963262, -79.3954612917379 43.6957424798034, -79.3955690097916 43.6957223523688, -79.39571629376429 43.6957001276303, -79.39588528740489 43.6956853665438, -79.3961081541657 43.6956666126055, -79.3962045104245 43.696127026043, -79.3964710412413 43.6973739370436, -79.3966427138492 43.6983012068135, -79.39686121045941 43.6993320241908, -79.3969539406084 43.6997866636074, -79.3970391853112 43.7002952858583, -79.3971592350693 43.7008534463644, -79.3972192468958 43.7011370226775, -79.3973441683584 43.7017941833586,

In [ ]:
# from shapely.geometry import Polygon
# import shapely.wkt

# # Iterate over each row in the DataFrame
# for index, row in neighbourhood_merged.iterrows():
#     # Convert the geometry string to a Shapely geometry object
#     geometry = shapely.wkt.loads(row['geometry'])
    
#     # Check if the geometry is valid
#     if not geometry.is_valid:
#         # If the geometry is invalid, attempt to fix it
#         geometry = geometry.buffer(0)
        
#         # Check if the geometry is now valid
#         if geometry.is_valid:
#             # If the geometry is valid after buffering, replace the original geometry
#             neighbourhood_merged.at[index, 'geometry'] = shapely.wkt.dumps(geometry)
#         else:
#             # If the geometry could not be fixed, handle the error accordingly
#             print(f"Unable to fix geometry at index {index}")


In [ ]:
neighbourhood_gdf.info()

In [ ]:
# Create spatial weights matrix (W) using lat2W method
W = lat2W(neighbourhood_gdf['geometry'])

# Convert the crime count to a numpy array
crime_count = neighbourhood_gdf['Crime_Count'].values

# Compute Moran's I
mi = Moran(crime_count, W)

# Print Moran's I statistic and p-value
print("Moran's I:", mi.I)
print("p-value:", mi.p_sim)


# Alternatively

# # Create the matrix of weights using lat2W
# W = lat2W(neighbourhood_gdf)

# # Create the Moran object
# mi = Moran(neighbourhood_gdf['Crime_Count'], W)

# # Print Moran's I results
# print(f"Moran's I: {mi.I:.4f}")
# print(f"p-value: {mi.p_norm:.4f}")
# moran_statistic = mi.I

In [ ]:

# # Create spatial weights matrix (W) using lat2W method
# W = lat2W(neighbourhood_merged['geometry'].y, neighbourhood_merged['geometry'].x)

# # Convert the crime count to a numpy array
# crime_count = neighbourhood_merged['Crime_Count'].values

# # Compute Moran's I
# moran = Moran(crime_count, W)

# # Print Moran's I statistic
# print("Moran's I:", moran.I)

# OR

# # Create a numpy array from the crime count column
# data_array = np.array(neighbourhood_gdf['Crime_Count'])

# # Create the matrix of weights (assuming you want to use lat2W)
# w = lat2W(neighbourhood_gdf.geometry)

# # Create the Moran object
# mi = Moran(data_array, w)

# # Print Moran's I results
# print(f"Moran's I: {mi.I:.4f}")
# print(f"p-value: {mi.p_norm:.4f}")
# moran_statistic = mi.I
